In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
data = pd.read_csv("forest_addFire_thresh0.csv")

In [3]:
data.head()
data=data.replace(-9999,np.nan)

In [4]:
Me2=data.loc[data["Site"]=='Me2']

In [5]:
from collections import Counter
def detect_outliers(df,n,features):
    outlier_indices = []
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        # outlier step
        outlier_step = 1.5 * IQR
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(Me2,2,["Net Radiation","Latent Heat","Sensible Heat","Soil Heat Flux","Air Temp",'Soil Temp','CO2 Flux'])

c:\users\ywang\appdata\local\programs\python\python36\lib\site-packages\numpy\lib\function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [6]:
Me2 = Me2.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

In [7]:
X=Me2[["Net Radiation","Latent Heat","Sensible Heat","Soil Heat Flux","Air Temp",'Soil Temp','Fire']]

In [8]:
y=Me2["CO2 Flux"]

In [9]:
def MaxMinNormalization(x,Max,Min):  
    x = (x - Min) / (Max - Min);  
    return x;  
X=MaxMinNormalization(X,np.max(X),np.min(X))
y=MaxMinNormalization(y,np.max(y),np.min(y))

In [10]:
bst = xgb.XGBRegressor(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,seed=1)
bst.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=6, missing=None, n_estimators=5000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, seed=1,
       silent=True, subsample=0.8)

In [11]:
bst.score(X, y, sample_weight=None)

0.95011537999904461

In [12]:
from sklearn import cross_validation

c:\users\ywang\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
scores = cross_validation.cross_val_score(bst, X, y, cv=10)#5-fold cv

In [14]:
scores

array([ 0.62663041,  0.54622904,  0.50130266,  0.59636331,  0.77278056,
        0.4272091 ,  0.71835753,  0.31819418,  0.64039983,  0.71962024])

In [15]:
from sklearn.metrics import r2_score

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4)

In [17]:
bst2 = xgb.XGBRegressor( 
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,seed=2)
bst2.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=6, missing=None, n_estimators=5000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1, seed=2,
       silent=True, subsample=0.8)

In [18]:
predictions = bst2.predict(X_test)

In [19]:
r2_score(predictions, y_test)

0.70169555314067988

In [20]:
bst2.score(X_test,y_test)

0.74948271307009628

In [20]:
predictions_all=bst.predict(X)

In [21]:
r2_score(predictions_all, y)

0.94305337463387806